In [1]:
%config IPCompleter.greedy=True
import os
import pygal
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import cairosvg
import datetime
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.stats import chisquare
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, roc_auc_score, roc_curve

In [3]:
path_datos = "/home/laloadan/Documentos/Diplomado_Ciencia_de_Datos/Modulo 2 Modelaciòn Supervisada/Bases de Datos/"
df = pd.read_csv(path_datos + "OnlineNewsPopularity.csv")

df.columns =[x.strip() for x in df.columns]

In [4]:
var_cont = ['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']


# --------------------------- Regresión Lineal -------------------------

In [5]:
# Se jugarà con las discretas para llegar a una variable objetivo continua
# El chiste es minimizar el error 
# Hay que calcular el coeficiente de relaciòn (r) y si es cercano a 1, ya la hicimos
var_disc = ['data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 
           'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'weekday_is_monday',
           'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
           'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend']


In [8]:
df['shares'].head(5)

0     593
1     711
2    1500
3    1200
4     505
Name: shares, dtype: int64

In [6]:
# Se declara una variable que almacenarà los mètodos y caracteristicas de la regresión lineal
mr = LinearRegression()

# mr se guarda el modelo, donde se necesita el array con las variables a jugar (X)
#  y la variable objetivo (y)
mr.fit(X = df[var_cont], y = df['shares'])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
# Se crea una variable "estimada" ŷ tomando en cuenta las variables continuas

df["y_hat"] = mr.predict(X = df[var_cont])

# Vemos que tal son los resultados que se calcularon
df[["shares", "y_hat"]].head(5)

,shares,y_hat
0,593,-245.539620
1,711,-806.713086
2,1500,742.545164
3,1200,-434.420744
4,505,-294.709800


Valió barriga... ¡¡¡Están bien culeros!!!

## Tal vez haya outlayer, hay que quitarlos

In [9]:
df_aux = pd.read_csv(path_datos + "OnlineNewsPopularity.csv")
df_aux.columns = [x.strip() for x in df_aux.columns]

df.shape

(39644, 62)

In [10]:
def New_pob_var_sigma(data_origen,var, enes=3):
    data = pd.DataFrame(data_origen)
    
    sigma = data[var].std()
    media = data[var].mean()
    lim_inf  = media - enes*sigma
    lim_sup = media + enes*sigma
    
    return lim_inf,lim_sup

In [11]:
# Un Dataframe es lo mismo que una tabla

for var_cont_i in var_cont:
# Se calculan las cotas superiores e inferiores de cada variable de la tabla original  (df)
# Asì no afectamos que se haya reducido la poblaciòn
    Cota_Inf, Cota_Sup = New_pob_var_sigma(df,var_cont_i,4)
    
# Se reducirà un dataframe auxiliar, tomando en cuenta los limites del dataframe original
    df_aux = df_aux[(Cota_Inf <= df_aux[var_cont_i]) & (df_aux[var_cont_i] <= Cota_Sup)]
  

df_aux.shape

(34307, 61)

In [13]:
# Hace falta quoitarlos de la variable objetivo
var_objetivo = "shares"

Cota_Inf, Cota_Sup = New_pob_var_sigma(df,var_objetivo,4)

df_aux = df_aux[(Cota_Inf <= df_aux[var_objetivo]) & (df_aux[var_objetivo] <= Cota_Sup)]
  

df_aux.shape


(34156, 61)

In [14]:
# A mì no me hace sentido quitar outlyers en las variables continuas si no se toman en cuenta
mr_aux = LinearRegression()
mr_aux.fit(X = df_aux[var_cont], y = df_aux['shares'])
df_aux["y_hat"] = mr_aux.predict(X = df_aux[var_cont])

# Vemos que tal son los resultados que se calcularon
df_aux[["shares", "y_hat"]].head(5)

,shares,y_hat
0,593,-136.339092
1,711,-257.403961
2,1500,550.320412
3,1200,-319.770683
5,855,263.979816


Aunque se redujo un poco la diferencia entre shares y la ŷ, sigue
estando bien culeros

## 1. Ajuste de la regresión lineal 

In [15]:
# Hay que ver la calificación que tiene el modelo
Score_original = mr.score(df[var_cont], df['shares'])
Score_aux = mr_aux.score(df_aux[var_cont], df_aux['shares'])

print(Score_original, Score_aux)

0.02258175402466578 0.06342531122250505


No hay mucha diferencia entre la tabla original y la auxiliar (Sin Outlyers). A fin de cuentas, estàn como para llorar

In [16]:
# Y ahora hay que ver que tanta relaciòn hay entre las variables. Entre màs cercano al 1 mejor.
r_original = r2_score(df['shares'], df['y_hat'])
r_aux = r2_score(df_aux['shares'], df_aux['y_hat'])

print(r_original, r_aux)

0.02258175402466578 0.06342531122250505


Sigue sin ser suficiente, la relaciòn es es menor al 10% en ambos casos

In [17]:
pema = (abs(df['shares'] - df['y_hat'])/df['shares'])*(1/df.shape[0])
pema_aux = (abs(df_aux['shares'] - df_aux['y_hat'])/df_aux['shares'])*(1/df_aux.shape[0])

print(sum(pema),sum(pema_aux))

1.9127664846961936 1.286827272852407


No sé que es pema, ni para que se usa o que significa ¡¡Maldita sea!!

## Encontrar las variables más significativas 

In [18]:
kb = SelectKBest(k=5, score_func=f_regression)
kb.fit(X=df[var_cont], y=df["shares"])
k_best = [x for x, y in zip([x for x in df.columns if x != "shares"], kb.get_support()) if y]

kb_aux = SelectKBest(k=5, score_func=f_regression)
kb_aux.fit(X=df_aux[var_cont], y=df_aux["shares"])
k_best_aux = [x for x, y in zip([x for x in df_aux.columns if x != "shares"], kb.get_support()) if y]


print(k_best)
print(k_best_aux)

['kw_min_min', 'kw_max_min', 'kw_max_max', 'kw_max_avg', 'kw_avg_avg']
['kw_min_min', 'kw_max_min', 'kw_max_max', 'kw_max_avg', 'kw_avg_avg']


/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:293: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Se encontraron las mismas variables significativas.

In [19]:
[(x,y) for x,y in zip(k_best,mr.coef_)]

[('kw_min_min', 1.7784121238524142),
 ('kw_max_min', 104.89866790374093),
 ('kw_max_max', 0.5398372364453004),
 ('kw_max_avg', 3805.882067584673),
 ('kw_avg_avg', -2063.72960667464)]

In [20]:
[(x_aux,y_aux) for x_aux,y_aux in zip(k_best_aux,mr_aux.coef_)]

[('kw_min_min', 0.5997353468300006),
 ('kw_max_min', 30.811442359077972),
 ('kw_max_max', 0.0577602408911356),
 ('kw_max_avg', -1030.4582642182768),
 ('kw_avg_avg', -22715793677.247406)]

La variable kw_avg_avg se disparó bien gacho

In [21]:
# Se vuelve a hacer la regresiòn con las nuevas variables y se saca su score
mr_2 = LinearRegression()
mr_2.fit(X = df[k_best], y = df['shares'])
score_1 = mr_2.score(df[k_best], df['shares'])

mr_2_aux = LinearRegression()
mr_2_aux.fit(X = df_aux[k_best_aux], y = df_aux['shares'])
score_2 = mr_2_aux.score(df_aux[k_best_aux], df_aux['shares'])

print(score_1,score_2)

0.014779226693894419 0.04366000490546973


Quedaron casi igual de culeros jajajaja

## Construcción de una variable discreta en función de la original 

In [29]:
# Lo que se puede hacer es crear una variable binaria donde sea 1 si es mayor al promedio y
# 0 en el resto de los casos

# Por defecto, nos regresa valores booleanos (T or F) entonces se cambia el tipo a entero
# Y nos quedaran 1 para T y 0 para F
df["tgt"] = (df["shares"] > df["shares"].mean()).astype(int)
print(df["shares"].mean())
df[["tgt","shares"]].head(15)


3395.3801836343455


,tgt,shares
0,0,593
1,0,711
2,0,1500
3,0,1200
4,0,505
5,0,855
6,0,556
7,0,891
8,1,3600
9,0,710


In [30]:
df_aux["tgt"] = (df_aux["shares"] > df_aux["shares"].mean()).astype(int)
print(df_aux["shares"].mean())
df_aux[["tgt","shares"]].head(15)

2749.935472537768


,tgt,shares
0,0,593
1,0,711
2,0,1500
3,0,1200
5,0,855
9,0,710
10,0,2200
11,0,1900
14,0,761
15,0,1600


# Regresión Logistica